In [1]:
import torch
import torch.nn as nn

#### Convolutional Block
Every convolution is followed by an activation function and batch normalization.

In [ ]:
Activations = nn.ModuleDict([
    ['relu', nn.ReLU()],
    ['identity', nn.Identity()]
])

def check_AutoPadding(kernel_size):
    if type(kernel_size) == tuple:
        return (kernel_size[0] // 2, kernel_size[1] // 2)
    else:
        return kernel_size // 2

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride = 1, padding = 0, use_autoPadding = False, use_batchNorm = False, activation = None):
        super(ConvBlock, self).__init__()
        
        padding = AutoPadding(kernel_size) if use_autoPadding else padding
        
        self.conv = nn.Conv2d(in_channels, out_channels,
                              kernel_size, stride, padding)
        self.batch_norm = nn.BatchNorm2d(out_channels) if use_batchNorm else Activations['identity']
        self.activation = Activations[activation] if activation != None else Activations['identity']
        
    def forward(self, X):
        return self.activation(self.batch_norm(self.conv(X)))

In [ ]:
class InceptionModule(nn.Module):
    def __init__(self, in_channels, f_1x1, f_3x3_r, f_3x3, f_5x5_r, f_5x5, f_pp):
        super(InceptionModule, self).__init__()
        
        self.branch1 = nn.Sequential(
            ConvBlock(in_channels, f_1x1, kernel_size = 1, stride = 1, padding = 0)
        )
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, f_3x3_r, kernel_size = 1, stride=  1, padding = 0),
            ConvBlock(f_3x3_r, f_3x3, kernel_size = 3, stride = 1, padding = 1)
        )
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, f_5x5_r, kernel_size = 1, stride = 1, padding = 0),
            ConvBlock(f_5x5_r, f_5x5, kernel_size = 5, stride = 1, padding = 2)
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(3, stride = 1, padding = 1, ceil_mode = True),
            ConvBlock(in_channels, f_pp, kernel_size = 1, stride = 1, padding = 0)
        )
    
    def forward(self, X):
        return torch.cat([
            self.branch1(X),
            self.branch2(X),
            self.branch3(X),
            self.branch4(X)
        ], 1)

__Adaptive average pooling__ is simply an average pooling operation that, given an input and output dimensionality, calculates the correct kernel size necessary to produce an output of the given dimensionality from the given input.

In [ ]:
class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        
        self.pool = nn.AdaptiveAvgPool2d((4, 4))
        self.conv = ConvBlock(in_channels, 128, kernel_size = 1, activation = 'relu', use_batchNorm = False)
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(1024, num_classes)
        
    def forward(self, X):
        X = self.pool(X)
        X = self.conv(X)
        X = torch.flatten(X, 1)
        X = self.fc1(X)
        X = self.dropout(X)
        X = self.fc2(X)
        return X

In [ ]:
class GoogleNet(nn.Module):
    def __init__(self, num_classes = 10):
        self.conv1 = ConvBlock(3, 64, kernel_size = 7, stride = 2, use_autoPadding = True, use_batchNorm = True, activation = 'relu'),
        self.pool1 = nn.MaxPool2d(3, stride = 2, padding = 0, ceil_mode = True),
        self.conv2 = ConvBlock(64, 64, kernel_size = 1, use_autoPadding = True, use_batchNorm = True, activation = 'relu'),
        self.conv3 = ConvBlock(64, 192, kernel_size = 3, use_autoPadding = True, use_batchNorm = True, activation = 'relu'),
        self.pool2 = nn.MaxPool2d(3, stride = 2, padding = 0, ceil_mode = True)
    
        self.inception3a = InceptionModule(in_channels = 192,
                                           f_1x1 = 64,
                                           f_3x3_r = 96,
                                           f_3x3 = 128,
                                           f_5x5_r = 16,
                                           f_5x5 = 32,
                                           f_pp = 32)
        self.inception3b = InceptionModule(in_channels = 256,
                                           f_1x1 = 128,
                                           f_3x3_r = 128,
                                           f_3x3 = 192,
                                           f_5x5_r = 32,
                                           f_5x5 = 96,
                                           f_pp = 64)
        self.pool4 = nn.MaxPool2d(3, stride = 2, padding = 0, ceil_mode = True)
        self.inception4a = InceptionModule(in_channesls = 480,
                                           f_1x1 = 192,
                                           f_3x3_r = 96,
                                           f_3x3 = 208,
                                           f_5x5_r = 16,
                                           f_5x5 = 48,
                                           f_pp = 64)
        self.inception4b = InceptionModule(in_channels = 512,
                                           f_1x1 = 160,
                                           f_3x3_r = 112,
                                           f_3x3 = 224,
                                           f_5x5_r = 24,
                                           f_5x5 = 64,
                                           f_pp = 64)
        self.inception4c = InceptionModule(in_channels = 512,
                                           f_1x1 = 128,
                                           f_3x3_r = 128,
                                           f_3x3 = 256,
                                           f_5x5_r = 24,
                                           f_5x5 = 64,
                                           f_pp = 64)
        self.inception4d = InceptionModule(in_channels = 528,
                                           f_1x1 = 112,
                                           f_3x3_r = 144,
                                           f_3x3 = 288,
                                           f_5x5_r = 32,
                                           f_5x5 = 64,
                                           f_pp = 64)
        self.pool5 = nn.MaxPool2d(3, stride = 2, padding = 0, ceil_mode = True)
        self.inception5a = InceptionModule(in_channels = 832,
                                           f_1x1 = 256,
                                           f_3x3_r = 160,
                                           f_3x3 = 320,
                                           f_5x5_r = 32,
                                           f_5x5 = 128,
                                           f_pp = 128)
        self.inception5b = InceptionModule(in_channels = 832,
                                           f_1x1 = 384,
                                           f_3x3_r = 192,
                                           f_3x3 = 384,
                                           f_5x5_r = 48,
                                           f_5x5 = 128,
                                           f_pp = 128)
        self.pool6 = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
        self.aux4a = InceptionAux(512, num_classes)
        self.aux4d = InceptionAux(528, num_classes)
    
    def forward(self, X):
        X = self.conv1(X)
        X = self.pool1(X)
        X = self.conv2(X)
        X = self.conv3(X)
        X = self.pool3(X)
        X = self.inception3a(X)
        X = self.inception3b(X)
        X = self.pool4(X)
        X = self.inception4a(X)
        
        aux1 = self.aux4a(X)
        
        X = self.inception4b(X)
        X = self.inception4c(X)
        X = self.inception4d(X)
        
        aux2 = self.aux4d(X)
        
        X = self.inception4e(X)
        X = self.pool5(X)
        X = self.inception5a(X)
        X = self.inception5b(X)
        X = self.pool6(X)
        X = self.flatten(X, 1)
        X = self.dropout(X)
        X = self.fc(X)
        
        return x, aux1, aux2
        